In [1]:
# Imports
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [2]:
# Read raw data
data = pd.read_csv("C:\\Users\\bkb3\\Documents\\US_Accidents_March23.csv")

In [ ]:
# Subset to MD based data only
df = data[data["State"]=='MD']


In [4]:
# View data, remove duplicate rows
df.head()
df = df.drop_duplicates()

In [5]:
# Check all unique values for all columns
for col in df.columns:
    print(df[col].value_counts(sort=True))

ID
A-7777383    1
A-7774726    1
A-7774727    1
A-7774733    1
A-7774736    1
            ..
A-228580     1
A-228630     1
A-228632     1
A-228641     1
A-228649     1
Name: count, Length: 140417, dtype: int64
Source
Source1    98441
Source2    41321
Source3      655
Name: count, dtype: int64
Severity
2    103558
3     25450
4     10892
1       517
Name: count, dtype: int64
Start_Time
2021-05-28 07:31:48    30
2022-05-18 07:05:30    22
2021-11-06 16:59:00    19
2022-04-23 18:32:30    17
2020-06-26 18:14:49    17
                       ..
2019-08-15 16:21:25     1
2019-08-15 18:25:50     1
2019-08-16 19:57:05     1
2019-08-16 19:59:23     1
2016-12-01 06:35:00     1
Name: count, Length: 117296, dtype: int64
End_Time
2022-07-15 15:23:55    10
2020-12-19 00:51:28    10
2021-02-22 12:48:14    10
2020-04-30 16:37:30     9
2021-05-28 09:11:52     9
                       ..
2019-08-15 16:50:26     1
2019-08-15 18:53:25     1
2019-08-16 20:26:17     1
2019-08-16 20:28:09     1
2016-11-30 17:0

In [6]:
# Worth keeping weather time stamp? Examine value types
print(df["Weather_Timestamp"].value_counts(sort=True))

Weather_Timestamp
2022-03-13 01:55:00    64
2021-05-28 07:58:00    34
2022-05-18 06:52:00    29
2021-02-04 14:56:00    26
2021-11-06 16:55:00    25
                       ..
2019-08-14 15:08:00     1
2019-08-14 15:23:00     1
2019-08-14 16:43:00     1
2019-08-10 09:54:00     1
2019-08-10 18:56:00     1
Name: count, Length: 70648, dtype: int64


In [7]:
# Check all values of all remaining cols
for col in df.columns:
    print(df[col].value_counts(sort=True))

ID
A-7777383    1
A-7774726    1
A-7774727    1
A-7774733    1
A-7774736    1
            ..
A-228580     1
A-228630     1
A-228632     1
A-228641     1
A-228649     1
Name: count, Length: 140417, dtype: int64
Source
Source1    98441
Source2    41321
Source3      655
Name: count, dtype: int64
Severity
2    103558
3     25450
4     10892
1       517
Name: count, dtype: int64
Start_Time
2021-05-28 07:31:48    30
2022-05-18 07:05:30    22
2021-11-06 16:59:00    19
2022-04-23 18:32:30    17
2020-06-26 18:14:49    17
                       ..
2019-08-15 16:21:25     1
2019-08-15 18:25:50     1
2019-08-16 19:57:05     1
2019-08-16 19:59:23     1
2016-12-01 06:35:00     1
Name: count, Length: 117296, dtype: int64
End_Time
2022-07-15 15:23:55    10
2020-12-19 00:51:28    10
2021-02-22 12:48:14    10
2020-04-30 16:37:30     9
2021-05-28 09:11:52     9
                       ..
2019-08-15 16:50:26     1
2019-08-15 18:53:25     1
2019-08-16 20:26:17     1
2019-08-16 20:28:09     1
2016-11-30 17:0

In [8]:
# Date/time conversion, calculate length of traffic holdup in hours/fractions of an hour
df['Start_Time'] = pd.to_datetime(df['Start_Time'],format='mixed')
df['End_Time'] = pd.to_datetime(df['End_Time'], format = 'mixed')
df['Accident_Time_Hr'] = (df.End_Time - df.Start_Time) / pd.Timedelta(hours=1)
df


,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Accident_Time_Hr
228546,A-228553,Source2,2,2016-11-30 11:58:01,2016-11-30 16:57:20,39.152046,-76.645584,NaN,NaN,0.010,...,False,False,False,False,False,Day,Day,Day,Day,4.988611
228547,A-228554,Source2,3,2016-11-30 12:04:03,2016-11-30 17:02:36,39.152046,-76.645584,NaN,NaN,0.010,...,False,False,False,False,False,Day,Day,Day,Day,4.975833
228560,A-228567,Source2,2,2016-11-30 16:54:53,2016-11-30 17:24:12,39.481083,-76.249626,NaN,NaN,0.010,...,False,False,False,True,False,Night,Day,Day,Day,0.488611
228567,A-228574,Source2,2,2016-11-30 17:33:47,2016-11-30 18:01:49,39.158756,-76.718475,NaN,NaN,0.010,...,False,False,False,False,False,Night,Night,Day,Day,0.467222
228573,A-228580,Source2,2,2016-11-30 17:35:33,2016-11-30 18:05:10,39.636391,-77.710457,NaN,NaN,0.010,...,False,False,False,True,False,Night,Night,Day,Day,0.493611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7727511,A-7776879,Source1,2,2019-08-22 19:48:26,2019-08-22 20:17:11,39.494560,-76.892370,39.49561,-76.89158,0.084,...,False,False,False,False,False,Day,Day,Day,Day,0.479167
7727512,A-7776880,Source1,2,2019-08-22 19:48:26,2019-08-22 20:17:11,39.495610,-76.891580,39.49456,-76.89237,0.084,...,False,False,False,True,False,Day,Day,Day,Day,0.479167
7727513,A-7776881,Source1,4,2019-08-22 19:57:25,2019-08-23 00:56:06,39.600500,-77.677230,39.60544,-77.73422,3.053,...,False,False,False,False,False,Night,Day,Day,Day,4.978056
7727996,A-7777364,Source1,2,2019-08-23 08:03:29,2019-08-23 08:32:31,39.336120,-76.491730,39.33962,-76.49409,0.273,...,False,False,False,False,False,Day,Day,Day,Day,0.483889


In [9]:
# Check unique values for all cols again
for col in df.columns:
    print(df[col].value_counts(sort=True))

ID
A-7777383    1
A-7774726    1
A-7774727    1
A-7774733    1
A-7774736    1
            ..
A-228580     1
A-228630     1
A-228632     1
A-228641     1
A-228649     1
Name: count, Length: 140417, dtype: int64
Source
Source1    98441
Source2    41321
Source3      655
Name: count, dtype: int64
Severity
2    103558
3     25450
4     10892
1       517
Name: count, dtype: int64
Start_Time
2021-05-28 07:31:48    34
2022-05-18 07:05:30    29
2022-04-23 18:32:30    24
2021-11-06 16:59:00    24
2022-07-15 14:05:49    22
                       ..
2019-08-15 16:21:25     1
2019-08-15 18:25:50     1
2019-08-16 19:57:05     1
2019-08-16 19:59:23     1
2016-12-01 17:58:44     1
Name: count, Length: 111577, dtype: int64
End_Time
2022-07-15 15:23:55    14
2020-12-19 00:51:28    10
2021-02-22 12:48:14    10
2021-05-28 09:11:52    10
2022-03-12 16:54:44    10
                       ..
2019-08-15 16:50:26     1
2019-08-15 18:53:25     1
2019-08-16 20:26:17     1
2019-08-16 20:28:09     1
2016-11-30 18:0

In [10]:
# Error checking of date time error I was getting over and over (One row had date and time recorded with higher
# granularity)
df.iloc[46456, :]

ID                                                               A-3710810
Source                                                             Source1
Severity                                                                 2
Start_Time                                             2022-05-12 23:39:31
End_Time                                               2022-05-13 00:54:33
Start_Lat                                                        38.804332
Start_Lng                                                       -76.906416
End_Lat                                                          38.804518
End_Lng                                                         -76.906239
Distance(mi)                                                         0.016
Description              Incident on ALLENTOWN RD near ALLENTOWN RD Dri...
Street                                                                 NaN
City                                                          Temple Hills
County                   

In [11]:
# Create year, month, and day columns out of date time
df['Accident_Year'], df['Accident_Month'], df['Accident_Day'] = df['Start_Time'].dt.year, df['Start_Time'].dt.month, df['Start_Time'].dt.day
df


,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Accident_Time_Hr,Accident_Year,Accident_Month,Accident_Day
228546,A-228553,Source2,2,2016-11-30 11:58:01,2016-11-30 16:57:20,39.152046,-76.645584,NaN,NaN,0.010,...,False,False,Day,Day,Day,Day,4.988611,2016,11,30
228547,A-228554,Source2,3,2016-11-30 12:04:03,2016-11-30 17:02:36,39.152046,-76.645584,NaN,NaN,0.010,...,False,False,Day,Day,Day,Day,4.975833,2016,11,30
228560,A-228567,Source2,2,2016-11-30 16:54:53,2016-11-30 17:24:12,39.481083,-76.249626,NaN,NaN,0.010,...,True,False,Night,Day,Day,Day,0.488611,2016,11,30
228567,A-228574,Source2,2,2016-11-30 17:33:47,2016-11-30 18:01:49,39.158756,-76.718475,NaN,NaN,0.010,...,False,False,Night,Night,Day,Day,0.467222,2016,11,30
228573,A-228580,Source2,2,2016-11-30 17:35:33,2016-11-30 18:05:10,39.636391,-77.710457,NaN,NaN,0.010,...,True,False,Night,Night,Day,Day,0.493611,2016,11,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7727511,A-7776879,Source1,2,2019-08-22 19:48:26,2019-08-22 20:17:11,39.494560,-76.892370,39.49561,-76.89158,0.084,...,False,False,Day,Day,Day,Day,0.479167,2019,8,22
7727512,A-7776880,Source1,2,2019-08-22 19:48:26,2019-08-22 20:17:11,39.495610,-76.891580,39.49456,-76.89237,0.084,...,True,False,Day,Day,Day,Day,0.479167,2019,8,22
7727513,A-7776881,Source1,4,2019-08-22 19:57:25,2019-08-23 00:56:06,39.600500,-77.677230,39.60544,-77.73422,3.053,...,False,False,Night,Day,Day,Day,4.978056,2019,8,22
7727996,A-7777364,Source1,2,2019-08-23 08:03:29,2019-08-23 08:32:31,39.336120,-76.491730,39.33962,-76.49409,0.273,...,False,False,Day,Day,Day,Day,0.483889,2019,8,23


In [12]:
# Convert street names to strings
df['Street'].astype(str)

228546              I-97 S
228547              I-97 S
228560     Philadelphia Rd
228567            MD-100 E
228573       E Antietam St
                ...       
7727511          Gamber Rd
7727512           Emory Rd
7727513             I-70 W
7727996            I-695 N
7728015            I-270 N
Name: Street, Length: 140417, dtype: object

In [13]:
# Due to hardware constraints, I manually figured out which roads were highways by exporting the data to CSV, opening in 
# Excel, and manually scanning the names of streets, making liberal use of find/replace

# This process is not perfect, many of the highway off/on ramp accidents were recorded only with a number/letter combo (12A, 62B, etc)
# And due to my level of skill and current code, I could not figure out how to add those values to the regex list without accidentally
# adding many non-highway roads that simply happened to include a letter or number in the name
# The snippets I did manage to find account for the vast majority of highways to my knowledge however
# List of highway name snippets
lst = ['I-', 'Capital Beltway', 'Baltimore Washington', ' Hwy', ' Pkwy', ' Trwy', 'US-', 'MD-',
       ' Gtwy', ' Byp', ' Brg', ' Thruway', ' Pike', ' Fwy', 'State Route', 'Inter County', 'Expy', 'A-B' ]

# Create highway yes/no column (NOT PERFECT)
df['Highway']=np.where(df['Street'].str.contains('|'.join(lst), na=False), True, False)

In [14]:
# Check how many highway/non highway values
df["Highway"].value_counts(sort=True)

Highway
True     91960
False    48457
Name: count, dtype: int64

In [15]:
# Proper formatting of county names (I took an educated guess that "Baltimore" referred to Balt. County and not City, this
# may not be correct)

def replace_misspelled(text):
    corrections = {'St Marys': "St. Mary's", 'Prince Georges': "Prince George's", 'Queen Annes': "Queen Anne's", 
                   "Baltimore (City)":"Baltimore City","Saint Marys":"St. Mary's", "Baltimore County":"Baltimore"}
    for word, correction in corrections.items():
        text = text.replace(word, correction)
    return text

df['County'] = df['County'].apply(replace_misspelled)

df["County"].value_counts(sort=True)

County
Prince George's         27143
Montgomery              22565
Baltimore               22374
Frederick               13056
Anne Arundel            11950
Baltimore City          10216
Harford                  6802
Howard                   6134
Washington               4848
Cecil                    2482
Queen Anne's             2436
Charles                  2037
Carroll                  1360
Talbot                   1179
Worcester                1154
Caroline                  908
Allegany                  770
St. Mary's                658
Wicomico                  621
Dorchester                593
Garrett                   360
Kent                      328
Calvert                   223
Somerset                  216
District Of Columbia        4
Name: count, dtype: int64

In [16]:
# One hot function (Could not find the one line function Dr. Olsen referred to)
def onehotinator(df, string):
    one_hot = pd.get_dummies(df[string], prefix=string)
    one_hot = one_hot.astype(int)
    df = df.join(one_hot)
    df = df.drop(columns=string)
    return df

In [17]:
# One hot encode counties
df = onehotinator(df, 'County')

In [18]:
# Boolean column feature for if snowy OR icy conditions were present
lst = ['Snow', 'Freezing', 'Ice', 'Wintry', 'Hail', 'Sleet']
df['Snow_Or_Ice']=np.where(df['Weather_Condition'].str.contains('|'.join(lst), na=False), True, False)

In [ ]:
# Boolean column feature for if mist OR fog OR haze was present, OR if visibility in miles was less than 0.001 mi
lst = ['Fog', 'Mist', 'Haze', 'Heavy']
df['Low_Vis_Weather']=np.where((df['Weather_Condition'].str.contains('|'.join(lst), na=False) | df["Visibility(mi)"] < 0.001), True, False)

In [20]:
# Boolean column feature for if road conditions were wet (weather was rain/storm related, OR there was more than 0 inches
# of precipitation at time of crash)
lst = ['Rain', 'Drizzle', 'Thunderstorm', 'T-Storm', 'Precipitation']
df['Rain']=np.where((df['Weather_Condition'].str.contains('|'.join(lst), na=False) | df['Precipitation(in)'] >= 0.01), True, False)

In [21]:
# Check which columns to delete
df.head()
df.columns

Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Street', 'City', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight', 'Accident_Time_Hr', 'Accident_Year',
       'Accident_Month', 'Accident_Day', 'Highway', 'County_Allegany',
       'County_Anne Arundel', 'County_Baltimore', 'County_Baltimore City',
       'County_Calvert', 'County_Caroline', 'County_Carroll', 'County_Cecil',
       'County_Charles', 'Coun

In [22]:
# Remove all but one day/night column
# Col measures were: 
# Sun up/down based on the time of day
# Sun up/down based on street lights being needed or not
# Sun up/down based on being able to see the horizon at sea
# Sun up/down based on being able to see stars
df = df.drop(columns=['Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Weather_Condition'])
df.columns


Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Street', 'City', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Accident_Time_Hr', 'Accident_Year', 'Accident_Month',
       'Accident_Day', 'Highway', 'County_Allegany', 'County_Anne Arundel',
       'County_Baltimore', 'County_Baltimore City', 'County_Calvert',
       'County_Caroline', 'County_Carroll', 'County_Cecil', 'County_Charles',
       'County_District Of Columbia', 'County_Dorchester', 'County_Frederick',
       'County_Garrett', 

In [23]:
# Reference for data conversion
df.dtypes

ID                          object
Source                      object
Severity                     int64
Start_Time          datetime64[ns]
End_Time            datetime64[ns]
                         ...      
County_Wicomico              int64
County_Worcester             int64
Snow_Or_Ice                   bool
Low_Vis_Weather               bool
Rain                          bool
Length: 74, dtype: object

In [24]:
# Convert all boolean columns to 1/0 integer columns and only day/night column to 1/0

bools = ['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 
             'Traffic_Signal', 'Turning_Loop', 'Highway', 'Snow_Or_Ice', 'Low_Vis_Weather', 'Rain']

for i in bools:
    df = df.astype({i:int}) 
    
df['Sunrise_Sunset'] = df.Sunrise_Sunset.map({'Day':1,'Night':0}) 

df.dtypes

ID                          object
Source                      object
Severity                     int64
Start_Time          datetime64[ns]
End_Time            datetime64[ns]
                         ...      
County_Wicomico              int64
County_Worcester             int64
Snow_Or_Ice                  int64
Low_Vis_Weather              int64
Rain                         int64
Length: 74, dtype: object

In [25]:
# Slowdown feature based on if 3 or more traffic speed slowing measures were present at accident

# (3 is an arbitrary number, 2 didn't feel significant in my gut, 4 barely returned any True)
def slowdown(row):
   if row['Bump'] + row["Crossing"] + \
   row["Give_Way"] + row["Junction"] + row["Railway"] + \
   row['Roundabout'] + row["Stop"] + row["Traffic_Calming"] +\
   row["Traffic_Signal"] + row["Turning_Loop"] >= 3:
      return 1
   else:
      return 0

df['Slowdown'] = df.apply(slowdown, axis=1)

df['Slowdown'].value_counts()

Slowdown
0    139955
1       462
Name: count, dtype: int64

In [26]:
# Reference for what to do next
df.isnull().sum()


ID                  0
Source              0
Severity            0
Start_Time          0
End_Time            0
                   ..
County_Worcester    0
Snow_Or_Ice         0
Low_Vis_Weather     0
Rain                0
Slowdown            0
Length: 75, dtype: int64

In [27]:
# Reference for feature engineering ideas
max(df['Temperature(F)']), min(df['Temperature(F)'])


(100.9, 1.0)

In [28]:
# Reference for feature engineering ideas
df['Wind_Chill(F)'].value_counts(sort=True)


Wind_Chill(F)
 70.0    2204
 73.0    2144
 72.0    2094
 52.0    2012
 54.0    1910
         ... 
 0.3        1
-4.0        1
 7.4        1
-6.9        1
 43.7       1
Name: count, Length: 517, dtype: int64

In [ ]:
# Column for potentially freezing conditions if temp at accident time was below freezing OT windchill was below freezing
# OR if the previous column checking for snow and ice was true snow/ice present

def freezing(row):
   if (row["Temperature(F)"]<=32) | (row["Wind_Chill(F)"]<=32) | (row["Snow_Or_Ice"] == 1):
      return 1
   else:
      return 0

df['Freezing?'] = df.apply(freezing, axis=1)

df["Freezing?"].value_counts()


Freezing?
0    121419
1     18998
Name: count, dtype: int64

In [30]:
# Column for driving drowsy for if the temp was above 69, AND humidity percent was above 30, 
# OR humidity was above 30 percent AND the sun had set
def hot_n_humid(row):
   if ((row["Temperature(F)"]>=70) & (row["Humidity(%)"]>=30)) | ((row["Humidity(%)"]>=30) & (row["Sunrise_Sunset"] == 0)):
      return 1
   else:
      return 0

df['Humid_Tired'] = df.apply(hot_n_humid, axis=1)

df["Humid_Tired"].value_counts()

Humid_Tired
1    71247
0    69170
Name: count, dtype: int64

In [31]:
# Column for if accident occurred in foggy weather AND the sun had set 

def cannot_see(row):
   if ((row["Low_Vis_Weather"]==1) & (row["Sunrise_Sunset"]==0)):
      return 1
   else:
      return 0

df['Cannot_See_At_Night'] = df.apply(cannot_see, axis=1)

df["Cannot_See_At_Night"].value_counts()

Cannot_See_At_Night
0    131544
1      8873
Name: count, dtype: int64

In [32]:
# Due to a technical ctrl+z error, I lost all instances of removing columns as I went, this is a quick fix (I didn't remember
# which columns were removed at which points)
df.isnull().sum()
df.columns

for col in df.columns:
    print(df[col].name, df[col].isnull().sum())

df = df.drop(columns=['ID','Source','Start_Time','End_Time','Start_Lat','Start_Lng','End_Lat','End_Lng','Weather_Timestamp',
           'Temperature(F)','Wind_Chill(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Direction',
           'Wind_Speed(mph)','Precipitation(in)','Description','Street','City','State','Zipcode','Country','Timezone',
           'Airport_Code'])

# Dropping remaining null values left

df = df.dropna(how='any')

df.isnull().sum()



ID 0
Source 0
Severity 0
Start_Time 0
End_Time 0
Start_Lat 0
Start_Lng 0
End_Lat 41976
End_Lng 41976
Distance(mi) 0
Description 0
Street 363
City 9
State 0
Zipcode 260
Country 0
Timezone 260
Airport_Code 327
Weather_Timestamp 13519
Temperature(F) 15713
Wind_Chill(F) 47596
Humidity(%) 15827
Pressure(in) 14528
Visibility(mi) 27461
Wind_Direction 20204
Wind_Speed(mph) 34300
Precipitation(in) 49483
Amenity 0
Bump 0
Crossing 0
Give_Way 0
Junction 0
No_Exit 0
Railway 0
Roundabout 0
Station 0
Stop 0
Traffic_Calming 0
Traffic_Signal 0
Turning_Loop 0
Sunrise_Sunset 286
Accident_Time_Hr 0
Accident_Year 0
Accident_Month 0
Accident_Day 0
Highway 0
County_Allegany 0
County_Anne Arundel 0
County_Baltimore 0
County_Baltimore City 0
County_Calvert 0
County_Caroline 0
County_Carroll 0
County_Cecil 0
County_Charles 0
County_District Of Columbia 0
County_Dorchester 0
County_Frederick 0
County_Garrett 0
County_Harford 0
County_Howard 0
County_Kent 0
County_Montgomery 0
County_Prince George's 0
County_Quee

Severity                       0
Distance(mi)                   0
Amenity                        0
Bump                           0
Crossing                       0
Give_Way                       0
Junction                       0
No_Exit                        0
Railway                        0
Roundabout                     0
Station                        0
Stop                           0
Traffic_Calming                0
Traffic_Signal                 0
Turning_Loop                   0
Sunrise_Sunset                 0
Accident_Time_Hr               0
Accident_Year                  0
Accident_Month                 0
Accident_Day                   0
Highway                        0
County_Allegany                0
County_Anne Arundel            0
County_Baltimore               0
County_Baltimore City          0
County_Calvert                 0
County_Caroline                0
County_Carroll                 0
County_Cecil                   0
County_Charles                 0
County_Dis

In [33]:

for col in df.columns:
    print(df[col].name, df[col].isnull().sum())

# Just to be safe, drop duplicates again
df = df.drop_duplicates()


Severity 0
Distance(mi) 0
Amenity 0
Bump 0
Crossing 0
Give_Way 0
Junction 0
No_Exit 0
Railway 0
Roundabout 0
Station 0
Stop 0
Traffic_Calming 0
Traffic_Signal 0
Turning_Loop 0
Sunrise_Sunset 0
Accident_Time_Hr 0
Accident_Year 0
Accident_Month 0
Accident_Day 0
Highway 0
County_Allegany 0
County_Anne Arundel 0
County_Baltimore 0
County_Baltimore City 0
County_Calvert 0
County_Caroline 0
County_Carroll 0
County_Cecil 0
County_Charles 0
County_District Of Columbia 0
County_Dorchester 0
County_Frederick 0
County_Garrett 0
County_Harford 0
County_Howard 0
County_Kent 0
County_Montgomery 0
County_Prince George's 0
County_Queen Anne's 0
County_Somerset 0
County_St. Mary's 0
County_Talbot 0
County_Washington 0
County_Wicomico 0
County_Worcester 0
Snow_Or_Ice 0
Low_Vis_Weather 0
Rain 0
Slowdown 0
Freezing? 0
Humid_Tired 0
Cannot_See_At_Night 0


In [34]:
df.shape
df.dtypes
# Somehow sun up/down col was a float, quick fix to be int
df["Sunrise_Sunset"]=df["Sunrise_Sunset"].astype(int)
df.dtypes
# Checked min and max of distance to decide on binning by hand or not
print(df['Distance(mi)'].min(),df['Distance(mi)'].max())


0.0 55.94600000000001


In [35]:
df.to_csv('Md_Data_Preprocessing_Draft2.csv', index=False)


In [40]:
# Reference for thinking about CFD based division (We have 4 classes, and CFD only takes 2 at a time)
df['Severity'].value_counts()



Severity
2    98633
3    24548
4     9676
1      457
Name: count, dtype: int64

In [39]:
df.columns

Index(['Severity', 'Distance(mi)', 'Amenity', 'Bump', 'Crossing', 'Give_Way',
       'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset',
       'Accident_Time_Hr', 'Accident_Year', 'Accident_Month', 'Accident_Day',
       'Highway', 'County_Allegany', 'County_Anne Arundel', 'County_Baltimore',
       'County_Baltimore City', 'County_Calvert', 'County_Caroline',
       'County_Carroll', 'County_Cecil', 'County_Charles',
       'County_District Of Columbia', 'County_Dorchester', 'County_Frederick',
       'County_Garrett', 'County_Harford', 'County_Howard', 'County_Kent',
       'County_Montgomery', 'County_Prince George's', 'County_Queen Anne's',
       'County_Somerset', 'County_St. Mary's', 'County_Talbot',
       'County_Washington', 'County_Wicomico', 'County_Worcester',
       'Snow_Or_Ice', 'Low_Vis_Weather', 'Rain', 'Slowdown', 'Freezing?',
       'Humid_Tired', 'Cannot_See_At_Night'],
  